In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt 

In [22]:
class VideoProcessor:
    def __init__(self, 
                 modelFile="../models/res10_300x300_ssd_iter_140000.caffemodel",
                 configFile = "../models/deploy.prototxt",
                 max_interations = 10,
                 conf_threshold = 0.60,
                 nframesdiff = 5,
                 normalized_dim = (256,256)):
        self.modelFile = modelFile
        self.configFile = configFile
        self.max_interations = max_interations
        self.conf_threshold = conf_threshold
        self.nframesdiff = nframesdiff
        self.normalized_dim = normalized_dim
        self.net = cv2.dnn.readNetFromCaffe(self.configFile, self.modelFile)
        self.mean = 128

    def extract_face(self, img):
        (h, w) = img.shape[:2]
        face = None
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (103.93, 116.77, 123.68))
        self.net.setInput(blob)
        detections = self.net.forward()
        for i in range(detections.shape[2]):
            if detections[0, 0, i, 2] > self.conf_threshold:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                face = img[y1:y2, x1:x2]
                face = cv2.resize(face, self.normalized_dim)
                break
                    
        return face

    def extract_random_faces(self, filename, num_faces):
        captured_faces = []
        iterations = 0
        v_cap = cv2.VideoCapture(filename)
        v_length = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        while len(captured_faces) < num_faces and iterations < self.max_interations:
            iterations += 1
            v_cap.set(1, np.random.randint(v_length)-1)
            
            ret, img = v_cap.read()
        
            if ret == True:
                (h, w) = img.shape[:2]
                blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (103.93, 116.77, 123.68))
                self.net.setInput(blob)
                detections = self.net.forward()
                for i in range(detections.shape[2]):
                    confidence = detections[0, 0, i, 2]            
                    if confidence > self.conf_threshold:
                        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                        (x1, y1, x2, y2) = box.astype("int")
                        face=img[y1:y2, x1:x2]
                        
                        # normlize
                        face = cv2.resize(face, self.normalized_dim)
                        captured_faces.append(face)
        
        # When everything done, release the video capture and video write objects
        v_cap.release()
    
        return captured_faces
  
    def extract_random_diff(self, filename, num_diff):
        captured_diff = np.empty(shape=(0,256,256,3), dtype=np.int8)
        iterations = 0
        v_cap = cv2.VideoCapture(filename)
        v_length = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        while len(captured_diff) < num_diff and iterations < self.max_interations:
            iterations += 1
            frame = np.random.randint(v_length)-1
            v_cap.set(1, frame)
            ret, img_base = v_cap.read()
            if ret == True:
                v_cap.set(1, frame + self.nframesdiff)
                ret, img = v_cap.read()
                
            if ret == True:
                img_base = cv2.cvtColor(img_base, cv2.COLOR_BGR2RGB)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                face1 = self.extract_face(img_base)
                face2 = self.extract_face(img)
                if type(face1) == type(face2):
                    face_diff = cv2.absdiff(face2,face1)
                    face_diff = cv2.absdiff(self.mean,face_diff)
                    if face_diff is not None:
                        captured_diff = np.append(captured_diff,[face_diff],axis=0)
        
        # When everything done, release the video capture and video write objects
        v_cap.release()
        
        # Closes all the frames
        cv2.destroyAllWindows() 

        return captured_diff

        

In [3]:
df = pd.DataFrame()
for root, dirs, files in os.walk('../videos', topdown=False):
    for name in dirs:
        print(name)
        dfdir = pd.read_json('../videos/' + name + '/metadata.json')
        dfdir = dfdir.T
        dfdir['dir'] = name
        df = df.append(dfdir)
df["processed"] = 'False'

df.to_csv('../data/metadata.csv',index_label='video')

dfdc_train_part_14
dfdc_train_part_48


In [4]:
df = pd.read_csv('../data/metadata.csv')

In [5]:
df.sample(10)

,video,label,original,split,dir,processed
4540,csgnemklmq.mp4,FAKE,irdptfgpbb.mp4,train,dfdc_train_part_48,False
4731,zwzlhvygxt.mp4,FAKE,vblnxpxxqc.mp4,train,dfdc_train_part_48,False
1288,ryshesrcep.mp4,FAKE,thvdoktlxs.mp4,train,dfdc_train_part_14,False
2603,jbxjqnwwhv.mp4,FAKE,nliofvksed.mp4,train,dfdc_train_part_48,False
1581,prswuwlrpw.mp4,FAKE,fdfyrgbtln.mp4,train,dfdc_train_part_14,False
1599,qtyqvmdsal.mp4,FAKE,hjoetslmwu.mp4,train,dfdc_train_part_14,False
4434,dqmrfhgpql.mp4,FAKE,xfyoksjdbf.mp4,train,dfdc_train_part_48,False
3208,cglpyitrzw.mp4,FAKE,baxpshhrhc.mp4,train,dfdc_train_part_48,False
2712,ypzbqkypyc.mp4,FAKE,jdxnotoadb.mp4,train,dfdc_train_part_48,False
3988,idwhtnofel.mp4,REAL,NaN,train,dfdc_train_part_48,False


In [ ]:
batch = 1
nsample = 100

vp = VideoProcessor()

while True:
    sample = df.query('label=="FAKE" and processed == False').sample(nsample)
    if len(sample) == 0:
        break

    fakes = np.empty(shape=(0,256,256,3), dtype=np.int8)
    reals = np.empty(shape=(0,256,256,3), dtype=np.int8)

    for index, row in tqdm(sample.iterrows(), total=nsample):
        diffs_fakes = vp.extract_random_diff('../videos/' + row.dir + '/' + row.video,2)
        fakes = np.append(fakes,diffs_fakes,axis=0)
        diffs_reals = vp.extract_random_diff('../videos/' + row.dir + '/' + row.original,2)
        reals = np.append(reals,diffs_reals,axis=0)

    np.savez(f'../data/train_{batch}', fakes=fakes, reals=reals)
    df.loc[sample.index,'processed'] = f'train_{batch}'
    df.to_csv('../data/metadata.csv',index = False)
    print(f'saved batch: {batch}')





  0%|          | 0/100 [00:00<?, ?it/s]



  1%|          | 1/100 [00:02<04:02,  2.45s/it]



  2%|▏         | 2/100 [00:05<04:09,  2.54s/it]



  3%|▎         | 3/100 [00:08<04:40,  2.89s/it]



  4%|▍         | 4/100 [00:12<04:47,  3.00s/it]



  5%|▌         | 5/100 [00:18<06:18,  3.99s/it]



  6%|▌         | 6/100 [00:24<06:59,  4.46s/it]



  7%|▋         | 7/100 [00:28<07:02,  4.54s/it]



  8%|▊         | 8/100 [00:39<10:02,  6.55s/it]



  9%|▉         | 9/100 [00:43<08:29,  5.60s/it]



 10%|█         | 10/100 [00:44<06:29,  4.33s/it]



 11%|█         | 11/100 [00:48<06:07,  4.13s/it]



 12%|█▏        | 12/100 [01:03<10:53,  7.42s/it]



 13%|█▎        | 13/100 [01:11<10:59,  7.58s/it]



 14%|█▍        | 14/100 [01:12<08:07,  5.67s/it]



 15%|█▌        | 15/100 [01:15<06:46,  4.78s/it]



 16%|█▌        | 16/100 [01:18<05:51,  4.19s/it]



 17%|█▋        | 17/100 [01:21<05:16,  3.81s/it]



 18%|█▊        | 18/100 [01:24<04:49,  3.53s/it]



 19%|█▉        | 19/100 [0

 27%|██▋       | 27/100 [01:49<04:07,  3.40s/it]



 28%|██▊       | 28/100 [01:52<03:50,  3.20s/it]



 29%|██▉       | 29/100 [02:01<05:43,  4.85s/it]



 30%|███       | 30/100 [02:04<05:04,  4.35s/it]



 31%|███       | 31/100 [02:07<04:28,  3.89s/it]



 32%|███▏      | 32/100 [02:08<03:36,  3.18s/it]



 33%|███▎      | 33/100 [02:12<03:47,  3.39s/it]



 34%|███▍      | 34/100 [02:17<04:15,  3.88s/it]



 35%|███▌      | 35/100 [02:18<03:18,  3.05s/it]



 36%|███▌      | 36/100 [02:21<03:05,  2.90s/it]



 37%|███▋      | 37/100 [02:24<03:07,  2.98s/it]



 38%|███▊      | 38/100 [02:27<03:01,  2.92s/it]



 39%|███▉      | 39/100 [02:29<02:46,  2.73s/it]



 40%|████      | 40/100 [02:37<04:07,  4.12s/it]



 41%|████      | 41/100 [02:38<03:14,  3.30s/it]



 42%|████▏     | 42/100 [02:40<02:56,  3.04s/it]



 43%|████▎     | 43/100 [02:43<02:47,  2.95s/it]



 44%|████▍     | 44/100 [02:45<02:29,  2.68s/it]



 45%|████▌     | 45/100 [02:48<02:24,  2.63s/it]



 46%|████▌  

In [ ]:
df.query('processed != False')